# Search Space에 대해

- 패키지 :`optuna.trial.Trial`
    - `suggest_categorical()`
    - `suggest_int()`
    - `suggest_float()`


In [1]:
import optuna
import optuna.trial.Trial as trial

In [2]:
def object(trial):
    optimizer = trial.suggest_categorical('optimizer', ['MomentumSGD', 'Adam'])
    
    num_layers = trial.suggest_int('num_layers', 1, 3)
    
    # Int에는 log도 넣을 수 있음
    # int에는 밑이 2인가?
    num_channels = trial.suggest_int('num_channels', 32, 512, log = True)
    
    # 이산형
    num_units = trial.suggest_int('num_units', 10, 100, step = 5)
    
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 1.0)
    
    # float에 쓰이는 log는 밑이 10인가?
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log = True)
    
    drop_path_rate = trial.suggest_float('drop_path_rate', 0.0, 1.0, step = 0.1)

- 사용 예시
- 파이썬 문법으로 `Branch`도 가능
- `Loop`도 가능

In [3]:
# Branch
import sklearn.ensemble
import sklearn.svm

def objective(trial):
    classifier_name = trial.suggest_categorical('classifier', ['svc', 'RandomForest'])
    if classifier_name == 'SVC':
        svc_c = trial.suggest_float('svc_c', 1e-10, 1e10, log = True)
        classifier_obj = sklearn.svm.SVC(C = svc_c)
        
    else:
        rf_max_depth = trial.suggest_int('rf_max_depth', 2, 32, log = True)
        classifier_obj = sklearn.ensemble.RandomForestClassifier(max_depth = rf_max_depth)

In [4]:
# Loop
import torch
import torch.nn as nn

def create_model(trial, in_size):
    n_layers = trial.suggest_int('n_layers', 1, 3)
    
    layers = []
    for i in range(n_layers):
        n_units : trial.suggest_int('n_units_l{}'.format(i), 4, 128, log = True)
        layers.append(nn.Linear(in_size, n_units))
        layers.append(nn.ReLU())
        in_size = n_units
        
    layers.append(nn.Linear(in_size, 10))
    
    return nn.Sequential(*layers)

ModuleNotFoundError: No module named 'torch'

- 최적화의 난이도는 파라미터의 수에 `Exponential`하게 증가함. 
- 따라서 중요하지 않은 파라미터는 제외하는 것이 좋다.